In [1]:
import geopandas
import json

# Define the initial CityJSON file
citymodel = "montreal.json"

# Define the roads dataset
roads_file = "roads.gpkg"

output = "montral_surfaces.json"

# Open files

Let's open the CityJSON file:

In [2]:
with open("montreal.json") as file:
    cm = json.load(file)

len(list(cm["CityObjects"]))

28018

Let's open the initial roads file:

In [3]:
roads = geopandas.read_file(roads_file)

len(roads)

4387

In [4]:
main_roads = roads[roads["road_type"] == "Main road"]
secondary_roads = roads[roads["road_type"] != "Main road"]

In [5]:
bgt_common_boundaries = geopandas.overlay(secondary_roads, main_roads, how='intersection', keep_geom_type=False)

In [6]:
bgt_common_boundaries['length'] = bgt_common_boundaries['geometry'].length

max_rows = bgt_common_boundaries[['ID_VOI_VOI_1', 'road_type_1', 'ID_VOI_VOI_2', 'geometry', 'length']].groupby('ID_VOI_VOI_1')['length'].idxmax()

In [7]:
temp_roads = bgt_common_boundaries.loc[max_rows]
temp_roads

,ID_VOI_VOI_1,road_type_1,ID_VOI_VOI_2,road_type_2,geometry,length
320,100055922,Cycle lane,100056227,Main road,"MULTILINESTRING ((298985.255 5042865.751, 2989...",18.702953
3022,100055934,Cycle lane,100056385,Main road,"MULTILINESTRING ((299480.182 5042276.370, 2994...",11.378621
2028,100055945,Cycle lane,100056271,Main road,"LINESTRING (299064.256 5043027.856, 299092.330...",33.605466
3001,100055947,Cycle lane,100056570,Main road,"MULTILINESTRING ((299472.809 5042265.083, 2994...",11.669484
2078,100055987,Cycle lane,100056761,Main road,"MULTILINESTRING ((299076.221 5042975.964, 2990...",14.551826
...,...,...,...,...,...,...
1798,250400692,Traffic island,200409701,Main road,"MULTILINESTRING ((299705.378 5043907.954, 2997...",52.386939
1736,250400694,Traffic island,200409724,Main road,"MULTILINESTRING ((299768.343 5044000.000, 2997...",53.306757
1735,250400695,Traffic island,200409724,Main road,"MULTILINESTRING ((299738.555 5043955.793, 2997...",38.128102
325,250401165,Traffic island,200133739,Main road,"MULTIPOLYGON (((298821.062 5042637.766, 298820...",1.697320


In [8]:
def append_parent(f):
    """Appends the parent geometry with this one as a semantic surface"""
    child_id = f["ID_VOI_VOI_1"]
    parent_id = f["ID_VOI_VOI_2"]
    
    parent = cm["CityObjects"][str(parent_id)]
    parent_geom = parent["geometry"][0]
    
    # Create a surface for the current surfaces
    if not "semantics" in parent_geom:
        parent_geom["semantics"] = {"surfaces": [{"type": "TrafficArea"}], "values": [0 for v in parent_geom["boundaries"]]}
    
    new_i = len(parent_geom["semantics"]["surfaces"])
    
    # Append new surfaces from child
    child = cm["CityObjects"][str(child_id)]
    child_geom = child["geometry"][0]
    
    parent_geom["semantics"]["surfaces"].append({"type": child[""]})
    
    for b in child_geom["boundaries"]:
        parent_geom["boundaries"].append(b)
        parent_geom["semantics"]["values"].append(new_i)
    
    # Delete child object
    del cm["CityObjects"][str(child_id)]
    
    print(parent)

temp_roads.apply(append_parent, axis=1)

KeyError: ''

In [ ]:
with open(output, "w") as out:
    json.dump(cm, out)